# Traveling Salesman Problem using Genetic Algorithm

In [1]:
%useLatestDescriptors
%use lets-plot

## Cities logic

In [2]:
import kotlin.random.Random

data class City(val x: Int, val y: Int, val id: String)

fun generateCities(numCities: Int): List<City> {
    val cities = mutableListOf<City>()
    val existingCoordinates = mutableSetOf<Pair<Int, Int>>()

    while (cities.size < numCities) {
        val x = Random.nextInt(0, 1000) 
        val y = Random.nextInt(0, 1000)

        val newPair = Pair(x, y)

        if (existingCoordinates.add(newPair)) {
            cities.add(City(x, y, cities.size.toString()))
        }
    }

    return cities
}

fun calculateDistance(city1: City, city2: City): Double {
    val dx = city2.x - city1.x
    val dy = city2.y - city1.y
    return sqrt(dx.toDouble() * dx + dy.toDouble() * dy)
} 

fun calculateHammingDistance(list1: List<City>, list2: List<City>): Int {
    var distance = 0
    
    for (i in 0 until list1.size) {
        val city1 = list1[i]
        val city2 = list2[i]
        if (city1.id != city2.id) {
            distance++
        }
    }
    
    return distance
}

## Population logic

In [17]:
data class Individual(val genes: List<City>) {
    override fun toString(): String {
        return genes.map { it.id }.toString()
    }
}

fun generateInitialPopulation(populationSize: Int, cities: List<City>): List<Individual> {
    val population = mutableListOf<Individual>()
    val existingRoutes = mutableSetOf<List<City>>()

    while (population.size < populationSize) {
        val shuffledCities = cities.shuffled() 
        val route = shuffledCities + shuffledCities[0]

        if (existingRoutes.add(route)) {
            population.add(Individual(route))
        }
    }

    return population
}

fun calculateTotalDistance(route: List<City>): Double {
    var sum = 0.0
    
    for (i in 0 until route.size - 1) {
        sum += calculateDistance(route[i], route[i+1])
    }
    
    return sum
}

fun selectParents(population: List<Individual>): Pair<Individual, Individual> {
    val first = population[Random.nextInt(population.size)]
    var max = -1
    var idxMax = -1
    
    for (i in population.indices) {
        val curr = calculateHammingDistance(first.genes, population[i].genes)
        if (curr >= max) {
            max = curr
            idxMax = i
        }
    }
    
    return Pair(first, population[idxMax])
}

fun buildCycle(first: Individual, second: Individual): List<Int> {
    require(first.genes.size == second.genes.size)

    val cycle = mutableListOf<Int>()

    var a = 0
    var b: City

    while (a !in cycle) {
        cycle.add(a)
        b = second.genes[a]
        a = first.genes.indexOf(b)
    }

    return cycle.toList().sorted()
}

fun cycleCrossover(first: Individual, second: Individual, mutationPercent: Int): Pair<Individual, Individual> {
    require(first.genes.size == second.genes.size)

    val cycle = buildCycle(first, second)

    val child1 = MutableList<City>(first.genes.size) {City(0, 0, "")}
    val child2 = MutableList<City>(first.genes.size) {City(0, 0, "")}

//    child1[0] = first.genes[0]
//    child2[0] = second.genes[0]

    for (i in 0 until first.genes.size) {
        if (i in cycle) {
            child1[i] = first.genes[i]
            child2[i] = second.genes[i]
        } else {
            child1[i] = second.genes[i]
            child2[i] = first.genes[i]
        }
    }
    
    var start = child1.first { it == child1.last() }
    child1[0] = start.also { child1[child1.indexOf(start)] = child1[0] }
    start = child2.first { it == child2.last() }
    child2[0] = start.also { child2[child2.indexOf(start)] = child2[0] }

    mutate(mutationPercent, child1)
    mutate(mutationPercent, child2)

    return Pair(Individual(child1), Individual(child2))
}

fun mutate(percent: Int, genes: MutableList<City>) {
    val num = Random.nextInt(0, 101)
    
    if (num <= percent) {
        val idx1 = Random.nextInt(1, genes.size - 1)
        var idx2 = Random.nextInt(1, genes.size - 1)
        
        while (idx1 == idx2) idx2 = Random.nextInt(1, genes.size - 1)
        
        genes[idx1] = genes[idx2].also { genes[idx2] = genes[idx1] }
    }
} 

fun generateNewPopulation(oldPopulation: MutableList<Individual>, mutationPercent: Int): List<Individual>{
    val newPopulation = mutableListOf(oldPopulation[0], oldPopulation[1]) // Elitism
    for (i in newPopulation) {
        oldPopulation.remove(i)
    }
    
    while (oldPopulation.size > 0) {
        val parents = selectParents(oldPopulation)
        oldPopulation.remove(parents.first)
        oldPopulation.remove(parents.second)
        
        val children = cycleCrossover(parents.first, parents.second, mutationPercent)
        newPopulation.add(children.first)
        newPopulation.add(children.second)
    }
    
    return newPopulation
}

## Genetic algorithm implementation

In [34]:
val citiesAmount = 30
val iterations = 1000
val mutationPercent = 10    // should be 0 <= 100

val cities = generateCities(citiesAmount)
var population = generateInitialPopulation(50, cities).sortedBy { calculateTotalDistance(it.genes) }

val bestRoutes = mutableListOf(population[0])
var df = mapOf("x" to cities.map { it.x }, "y" to cities.map { it.y }, 
                "start" to (0 until cities.size).map { it == 0 },
                "label" to cities.map { it.id })
var p = letsPlot(df) {x = "x"; y = "y"; label = "label"}

val bestRand = population[0]
println("Random generated population \t Best solution: ${calculateTotalDistance(bestRand.genes)}")
(p + geomPoint(df, showLegend = false) {color = "start";}  + geomLabel(hjust = 0, vjust = 1) {label = df[label]}).show()

for (i in 0 until iterations) {
    population = generateNewPopulation(population.toMutableList(), mutationPercent).sortedBy { calculateTotalDistance(it.genes) }
    bestRoutes.add(population[0])
    
    println("Iteration ${i + 1} \t Best solution: ${calculateTotalDistance(population[0].genes)}")
    
    if (abs(calculateTotalDistance(bestRoutes[bestRoutes.size - 1].genes) - 
    calculateTotalDistance(bestRoutes[bestRoutes.size - 2].genes)) < delta) {
        count += 1
    }
}

/*for (i in bestRoutes.indices) {
    df = mapOf("x" to bestRoutes[i].genes.map { it.x }, "y" to bestRoutes[i].genes.map { it.y }, 
                "start" to (0 until bestRoutes[i].genes.size).map { it == 0 },
                "label" to bestRoutes[i].genes.map { it.id })
    
    (p + geomPoint(df, showLegend = false) {color = "start";}  
    + geomLabel(hjust = 0, vjust = 1) {label = df[label]} 
    + geomPath(df)).show()
}*/

val bestRoute = bestRoutes.sortedBy { calculateTotalDistance(it.genes) }[0]
println("Best route overall: $bestRoute")
println("Total distance of best route: ${calculateTotalDistance(bestRoute.genes)}")
println("Improvement comparing to the first one: ${calculateTotalDistance(bestRand.genes) - calculateTotalDistance(bestRoute.genes)}")
df = mapOf("x" to bestRoute.genes.map { it.x }, "y" to bestRoute.genes.map { it.y }, 
                "start" to (0 until bestRoute.genes.size).map { it == 0 },
                "label" to bestRoute.genes.map { it.id })
    
    (p + geomPoint(df, showLegend = false) {color = "start";}  
    + geomLabel(hjust = 0, vjust = 1) {label = df[label]} 
    + geomPath(df)).show()

Random generated population 	 Best solution: 13826.324559173854
Iteration 1 	 Best solution: 13023.868667997642
Iteration 2 	 Best solution: 13023.868667997642
Iteration 3 	 Best solution: 13023.868667997642
Iteration 4 	 Best solution: 13023.868667997642
Iteration 5 	 Best solution: 12629.853701741175
Iteration 6 	 Best solution: 12629.853701741175
Iteration 7 	 Best solution: 12629.853701741175
Iteration 8 	 Best solution: 12629.853701741175
Iteration 9 	 Best solution: 12629.853701741175
Iteration 10 	 Best solution: 12629.853701741175
Iteration 11 	 Best solution: 12629.853701741175
Iteration 12 	 Best solution: 12629.853701741175
Iteration 13 	 Best solution: 12629.853701741175
Iteration 14 	 Best solution: 12629.853701741175
Iteration 15 	 Best solution: 12629.853701741175
Iteration 16 	 Best solution: 12629.853701741175
Iteration 17 	 Best solution: 12629.853701741175
Iteration 18 	 Best solution: 12629.853701741175
Iteration 19 	 Best solution: 12520.438102573362
Iteration 20 	

Improvement comparing to the first one: 3147.1328330855904


In [33]:
println("Best route overall: $bestRoute")
println("Total distance of best route: ${calculateTotalDistance(bestRoute.genes)}")
println("Improvement comparing to the first one: ${calculateTotalDistance(bestRand.genes) - calculateTotalDistance(bestRoute.genes)}")

Best route overall: [7, 20, 14, 8, 0, 21, 2, 17, 11, 5, 6, 16, 12, 22, 3, 26, 15, 9, 18, 10, 13, 19, 23, 4, 28, 29, 25, 24, 27, 1, 7]
Total distance of best route: 10840.28839829954
Improvement comparing to the first one: 2101.546138702599
